# Teleportation Kata

**Teleportation** quantum kata is a series of exercises designed to get you familiar with programming in Q#. It covers the quantum teleportation protocol which allows you to communicate a quantum state using only classical communication and previously shared quantum entanglement.

 - Teleportation is described in [this Wikipedia article](https://en.wikipedia.org/wiki/Quantum_teleportation).
 - An interactive demonstration can be found [on the Wolfram Demonstrations Project](http://demonstrations.wolfram.com/QuantumTeleportation/).

Each task is wrapped in one operation preceded by the description of the task.  Your goal is to fill in the blank (marked with `// ...` comment) with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

## Part I. Standard Teleportation
We split the teleportation protocol into several steps:
    
* Preparation (creating the entangled pair of qubits that are sent to Alice and Bob).
* Sending the message (Alice's task): Entangling the message qubit with Alice's qubit and extracting two classical bits to be sent to Bob.
* Reconstructing the message (Bob's task): Using the two classical bits Bob received from Alice to get Bob's qubit into the state in which the message qubit had been originally.  Finally, we compose these steps into the complete teleportation protocol.

### Task 1.1. Entangled pair

**Input:** two qubits qAlice and qBob, each in $|0\rangle$ state.

**Goal:**  prepare a Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ on these qubits.

> In the context of the quantum teleportation protocol, this is the preparation step: qubits qAlice and qBob will be sent to Alice and Bob, respectively.

In [1]:
%kata T11_Entangle 

operation Entangle (qAlice : Qubit, qBob : Qubit) : Unit {
    H(qAlice);
    CNOT(qAlice, qBob); 
}

Success!

### Task 1.2. Send the message (Alice's task)

Entangle the message qubit with Alice's qubit and extract two classical bits to be sent to Bob.

**Inputs**: 
1. Alice's part of the entangled pair of qubits qAlice.
2. The message qubit qMessage.

**Output**:
Two classical bits Alice will send to Bob via classical channel as a tuple of Bool values. The first bit in the tuple should hold the result of measurement of the message qubit, the second bit - the result of measurement of Alice's qubit.
Represent measurement result 'One' as `true` and 'Zero' as `false`.  The state of the qubits in the end of the operation doesn't matter.

In [2]:
%kata T12_SendMessage 

operation SendMessage (qAlice : Qubit, qMessage : Qubit) : (Bool, Bool) {
    CNOT(qMessage, qAlice);
    H(qMessage);
    return (M(qMessage) == One, M(qAlice) == One);
}

Success!

### Task 1.3. Reconstruct the message (Bob's task)

Transform Bob's qubit into the required state using the two classical bits received from Alice.

**Inputs**: 
1. Bob's part of the entangled pair of qubits qBob.
2. The tuple of classical bits received from Alice, in the format used in task 1.2.

**Goal** : Transform Bob's qubit qBob into the state in which the message qubit had been originally.

In [3]:
%kata T13_ReconstructMessage 

operation ReconstructMessage (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    if (not b1) {
        if (not b2) {
            // Do nothing.
        }
        else {
            X(qBob);
        }
    }
    else
    {
        if (not b2) {
            Z(qBob);
        }
        else {
            Z(qBob);
            X(qBob);
        }
    }
}

Success!

### Task 1.4. Standard teleportation protocol

Put together the steps implemented in tasks 1.1 - 1.3 to implement the full teleportation protocol.

**Inputs:**

1. The two qubits qAlice and qBob in $|0\rangle$ state.
2. The message qubit qMessage in the state $|\psi\rangle$ to be teleported.

**Goal:** Transform Bob's qubit qBob into the state $|\psi\rangle$. The state of the qubits qAlice and qMessage in the end of the operation doesn't matter.

In [4]:
%kata T14_StandardTeleport

operation StandardTeleport (qAlice : Qubit, qBob : Qubit, qMessage : Qubit) : Unit {
    Entangle(qAlice, qBob);
    let result = SendMessage(qAlice, qMessage);
    ReconstructMessage(qBob, result);
}

Success!

### Task 1.5. Prepare a state and send it as a message (Alice's task)

Given a Pauli basis along with a state `true` as 'One' or `false` as 'Zero', prepare a message qubit, entangle it with Alice's qubit, and extract two classical bits to be sent to Bob.

**Inputs:** 

1. Alice's part of the entangled pair of qubits qAlice.
2. A PauliX, PauliY, or PauliZ basis in which the message qubit should be prepared
3. A Bool indicating the eigenstate in which the message qubit should be prepared

**Output:** 

Two classical bits Alice will send to Bob via classical channel as a tuple of Bool values.  The first bit in the tuple should hold the result of measurement of the message qubit, the second bit - the result of measurement of Alice's qubit.  Represent measurement result 'One' as `true` and 'Zero' as `false`. The state of the qubit qAlice in the end of the operation doesn't matter.

In [5]:
%kata T15_PrepareAndSendMessage

open Microsoft.Quantum.Preparation;

operation PrepareAndSendMessage (qAlice : Qubit, basis : Pauli, state : Bool) : (Bool, Bool) {
    use msg = Qubit();
    if (state) {
        X(msg);
    }
    
    PreparePauliEigenstate(basis, msg);
    
    return SendMessage(qAlice, msg);
}

Success!

### Task 1.6. Reconstruct and measure the message state (Bob's task)

Transform Bob's qubit into the required state using the two classical bits received from Alice and measure it in the same basis in which she prepared the message.
    
**Inputs:**

1. Bob's part of the entangled pair of qubits qBob.
2. The tuple of classical bits received from Alice, in the format used in task 1.5.
3. The PauliX, PauliY, or PauliZ basis in which the message qubit was originally prepared.

**Output:** A Bool indicating the eigenstate in which the message qubit was prepared, 'One' as `true` and 'Zero' as `false`.  The state of the qubit qBob in the end of the operation doesn't matter.

> To get the output, transform Bob's qubit qBob into the state in which the message qubit was originally prepared, then measure it. 

In [6]:
%kata T16_ReconstructAndMeasureMessage

operation ReconstructAndMeasureMessage (qBob : Qubit, (b1 : Bool, b2 : Bool), basis : Pauli) : Bool {
    ReconstructMessage(qBob, (b1, b2));
    return Measure([basis], [qBob]) == Zero ? false | true;
}

Success!

### Task 1.7. Testing standard quantum teleportation

**Goal:** Test that the `StandardTeleport` operation from task 1.4 is able to successfully teleport the states $|0\rangle$ and $|1\rangle$, as well as superposition states such as $\frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big)$, $\frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big)$, $\frac{1}{\sqrt{2}} \big(|0\rangle + i|1\rangle\big)$ and $\frac{1}{\sqrt{2}} \big(|0\rangle - i|1\rangle\big)$.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_StandardTeleport` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_StandardTeleport`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details>
  <summary><b>Need a hint? Click here</b></summary>
  You may find your answers for 1.5 and 1.6 useful
</details>

In [7]:
open Microsoft.Quantum.Diagnostics;

open Microsoft.Quantum.Convert;

operation StandardTeleport_Impl(basis: Pauli, eigen: Bool) : Unit {
    use qMsg = Qubit();
    
    if (eigen) {
        X(qMsg);
    }
    
    PreparePauliEigenstate(basis, qMsg);
    
    DumpMachine();
    
    use qAlice = Qubit();
    use qBob = Qubit();
    
    Entangle(qAlice, qBob);
    
    let (msg, alice) = SendMessage(qAlice, qMsg);
    let result = ReconstructAndMeasureMessage(qBob, (msg, alice), basis);
    Message(BoolAsString(result));
}

StandardTeleport_Impl

In [8]:
open Microsoft.Quantum.Diagnostics;

open Microsoft.Quantum.Convert;

operation Run_StandardTeleport () : String {
    Message("Sending PauliX false:");
    StandardTeleport_Impl(PauliX, false);
    
    Message("Sending PauliX true:");
    StandardTeleport_Impl(PauliX, true);
    
    Message("Sending PauliY false:");
    StandardTeleport_Impl(PauliY, false);
    
    Message("Sending PauliY true:");
    StandardTeleport_Impl(PauliY, true);
    
    Message("Sending PauliZ false:");
    StandardTeleport_Impl(PauliZ, false);
    
    Message("Sending PauliZ true:");
    StandardTeleport_Impl(PauliZ, true);
    
    return "Success!";
}

Run_StandardTeleport

In [9]:
%simulate Run_StandardTeleport

Sending PauliX false:


|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

False
Sending PauliX true:


|0⟩	0.7071067811865476 + 0𝑖
|1⟩	-0.7071067811865476 + 0𝑖

True
Sending PauliY false:


...

|0⟩	-1 + 0𝑖
|1⟩	0 + 0𝑖

False
Sending PauliZ true:


|0⟩	0 + 0𝑖
|1⟩	-1 + 0𝑖

True


Success!

### Task 1.8. Entanglement swapping

Alice and Bob, independently from each other, each hold an entangled qubit pair in the state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big)$. They hand off one part of their pair to Charlie.

Charlie can now teleport the state of Alice's qubit he holds onto Bob's remaining qubit, thus teleporting the entanglement. 
Just like in "standard" teleportation, Bob still needs to apply the reconstruction steps - based on Charlie's measurement results - to the other qubit in his possession.

After this procedure the state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big)$ now spans across Alice's and Bob's qubits which they didn't send to Charlie. They are now maximally entangled, even though they never interacted in the first place!

**Outputs:**

A tuple of two operations.

The first operation is Charlie's part of the protocol. It will take two qubits as input (the ones Alice and Bob sent to Charlie), and produce a message, encoded as an integer, that will be sent to Bob.

The second operation is Bob's part of the protocol. It will take the qubit that remained in Bob's possession and Charlie's integer as input, and use the integer to adjust the state of Bob's qubit, so that Alice's and Bob's qubits end up in the state $|\Phi^{+}\rangle$.

> Note that you will likely need to create two separate helper operations that implement the two parts of the protocol,
 and return them, rather than implement the solution in the body of this operation.

<details>
  <summary><b>Need a hint? Click here</b></summary>
  You may find your answers for 1.2 and 1.3 useful, as similar steps are needed here.
</details>

In [10]:
operation EntanglementSwappingCharlie(qAlice : Qubit, qBob : Qubit) : Int {
    CNOT(qAlice, qBob);
    H(qAlice);
    
    if (M(qAlice) == Zero) {
        return M(qBob) == Zero ? 0 | 1;
    }
    else
    {
        return M(qBob) == Zero ? 2 | 3;
    }
}

EntanglementSwappingCharlie

In [11]:
operation EntanglementSwappingBob(qBob: Qubit, charlie: Int) : Unit {
    // I THINK that we have the following states for Alice and Bob's remaining Qubits based on measurement:
    // Zero, Zero => 1/sqrt(2)(|00> + |11>)
    // Zero, One => 1/sqrt(2)(|01> + |10>)
    // One, Zero => 1/sqrt(2)(|00> - |11>)
    // One, One => 1/sqrt(2)(|01> - |10>)
    // If charlie = 0, do nothing
    if (charlie == 1) {
        X(qBob);
    }
    
    if (charlie == 2) {
        Z(qBob);
    }
    
    if (charlie == 3) {
       X(qBob);
       Z(qBob);
    }
}

EntanglementSwappingBob

In [12]:
%kata T18_EntanglementSwapping

operation EntanglementSwapping () : ((Qubit, Qubit) => Int, (Qubit, Int) => Unit) {
    return (EntanglementSwappingCharlie, EntanglementSwappingBob);
}

Success!

## Part II. Teleportation using different entangled pair

In this section we will take a look at the changes in the reconstruction process (Bob's task) if the qubits shared between Alice and Bob are entangled in a different state.  Alice's part of the protocol remains the same in all tasks.  
As a reminder, the standard teleportation protocol requires shared qubits in state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big)$.

In each task, the inputs are
1. Bob's part of the entangled pair of qubits qBob.
2. The tuple of classical bits received from Alice, in the format used in task 1.2.

The goal is to transform Bob's qubit qBob into the state in which the message qubit had been originally.

### Task 2.1. Reconstruct the message if the entangled qubits were in the state $|\Phi^{-}\rangle = \frac{1}{\sqrt{2}} \big(|00\rangle - |11\rangle\big)$

In [13]:
%kata T21_ReconstructMessage_PhiMinus

operation ReconstructMessage_PhiMinus (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    //if (not b1) {
    //    if (not b2) {
            // 00 => alpha|0> - beta|1>
    //        Z(qBob);
    //    }
    //    else
    //    {
            // 01 => beta|0> - alpha|1>
    //        Z(qBob);
    //        X(qBob);
    //    }
    //}
    //else {
        // if not b2, then qBob should already be in correct state.
    //    if (b2) {
            // -beta|0> - alpha|1>
    //        Z(qBob);
    //       X(qBob);
    //        Z(qBob);
    //    }
    //}
    if (not b1 or b2) {
        Z(qBob);
    }
    
    if (b2) {
        X(qBob);
    }
    
    if (b1 and b2) {
        Z(qBob);
    }
}

Success!

### Task 2.2. Reconstruct the message if the entangled qubits were in the state $|\Psi^{+}\rangle = \frac{1}{\sqrt{2}} \big(|01\rangle + |10\rangle\big)$

In [14]:
%kata T22_ReconstructMessage_PsiPlus

operation ReconstructMessage_PsiPlus (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    //if (not b1) {
    //    if (not b2) {
            // beta|0> + alpha|1>
    //        X(qBob);
    //    }
        // if b2, then qBob is already in correct state.
    //}
    //else {
    //    if (not b2) {
            // alpha|1> - beta|0>
    //        X(qBob);
    //        Z(qBob);
    //    }
    //    else {
            // alpha|0> - beta|1>
    //        Z(qBob);
    //    }
    //}
    if (not b2) {
        X(qBob);
    }
    
    if (b1) {
        Z(qBob);
    }
}

Success!

### Task 2.3. Reconstruct the message if the entangled qubits were in the state $|\Psi^{-}\rangle = \frac{1}{\sqrt{2}} \big(|01\rangle - |10\rangle\big)$

In [15]:
%kata T23_ReconstructMessage_PsiMinus

operation ReconstructMessage_PsiMinus (qBob : Qubit, (b1 : Bool, b2 : Bool)) : Unit {
    //if (not b1) {
    //    if (not b2) {
            // -beta|0> + alpha|1>
    //        X(qBob);
    //        Z(qBob);
    //    }
    //    else {
            // -alpha|0> + beta|1>
    //        X(qBob);
    //        Z(qBob);
    //        X(qBob);
    //    }
    //}
    //else {
    //    if (not b2) {
            // alpha|1> + beta|0>
    //        X(qBob);
    //    }
    //    else {
            // -alpha|0> - beta|1>
    //        Z(qBob);
    //        X(qBob);
    //        Z(qBob);
    //        X(qBob);
    //    }
    //}
    if (b1 and b2) {
        Z(qBob);
    }
    
    X(qBob);
    
    if (not b1 or b2) {
        Z(qBob);
    }
    
    if (b2) {
        X(qBob);
    }
}

Success!

## Part III. Principle of deferred measurement

The principle of deferred measurement claims that measurements can be moved from an intermediate stage of a quantum circuit to the end of the circuit.  If the measurement results are used to perform classically controlled operations, they can be replaced by controlled quantum operations.

### Task 3.1. Measurement-free teleportation.

In this task we will apply this principle to the teleportation circuit.

**Inputs:** 

1. The two qubits qAlice and qBob in $|\Phi^{+}\rangle$ state.
2. The message qubit qMessage in the state $|\psi\rangle$ to be teleported.

**Goal:** transform Bob's qubit qBob into the state $|\psi\rangle$ using no measurements.  At the end of the operation qubits qAlice and qMessage should not be entangled with qBob.

In [16]:
%kata T31_MeasurementFreeTeleport

operation MeasurementFreeTeleport (qAlice : Qubit, qBob : Qubit, qMessage : Qubit) : Unit {
    // Entangle Message qubit and Alice's qubit.
    CNOT(qMessage, qAlice);
    H(qMessage);
    
    // Now instead of measuring qMessage and qAlice, send qBob through controlled gates
    // based on the possible results of this deferred measurement.
    Controlled X([qAlice], qBob);
    Controlled Z([qMessage], qBob);
}

Success!

## Part IV. Teleportation with three entangled qubits

Quantum teleportation using entangled states other than Bell pairs is also feasible.  Here we look at just one of many possible schemes - in it a state is transferred from Alice to a third participant Charlie, but this may only be accomplished if Charlie
has the trust of the second participant Bob.

### Task 4.1. Entangled trio

**Inputs:** three qubits qAlice, qBob, and qCharlie, each in $|0\rangle$ state.

**Goal:** create an entangled state $|\Psi^{3}\rangle = \frac{1}{2} \big(|000\rangle + |011\rangle + |101\rangle + |110\rangle\big)$ on these qubits.

In the context of the quantum teleportation protocol, this is the preparation step: qubits qAlice, qBob, and qCharlie will be sent to Alice, Bob, and Charlie respectively.

In [18]:
%kata T41_EntangleThreeQubits

operation EntangleThreeQubits (qAlice : Qubit, qBob : Qubit, qCharlie : Qubit) : Unit {
    //       1/2(|000> + |011> + |100> + |111>)
    // Goal: 1/2(|000> + |011> + |101> + |110>)
    
    // Get qBob and qCharlie into first Bell State.
    // |000>
    Entangle(qBob, qCharlie); // 1/sqrt(2)(|000> + |010>)
                              // 1/sqrt(2)(|000> + |011>)
    H(qAlice);                // 1/sqrt(2)(|000> + |100> + |011> + |111>)
                              // 1/sqrt(2)(|000> + |011> + |100> + |111>)
    CNOT(qAlice, qCharlie);   // 1/sqrt(2)(|000> + |011> + |101> + |110>)
}

Success!

### Task 4.2. Reconstruct the message (Charlie's task)

Alice has a message qubit in the state $|\psi\rangle$  to be teleported, she has entangled it with her own qubit from $|\Psi^{3}\rangle$ in the same manner as task 1.2 and extracted two classical bits in order to send them to Charlie. Bob has also measured his own qubit from $|\Psi^{3}\rangle$ and sent Charlie the result. Transform Charlie's qubit into the required state using the two classical bits received from Alice, and the one classical bit received from Bob.

**Inputs:** 
1. Charlie's part of the entangled trio of qubits qCharlie.
2. The tuple of classical bits received from Alice, in the format used in task 1.2.
3. A classical bit resulting from the measurement of Bob's qubit.

**Goal:** Transform Charlie's qubit qCharlie into the state in which the message qubit had been originally.

In [ ]:
%kata T42_ReconstructMessageWhenThreeEntangledQubits

operation ReconstructMessageWhenThreeEntangledQubits (qCharlie : Qubit, (b1 : Bool, b2 : Bool), b3 : Bool) : Unit {
    // ...
}